<a href="https://colab.research.google.com/github/hungryjins/Graph_DB/blob/main/Neo4j_Graph_Recommendation_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Setting

In [1]:
!pip install neo4j-graphrag neo4j openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 1

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

## 1. GraphDB




Neo4j Sandbox : https://sandbox.neo4j.com/

### 1-1. Neo4j Driver setting

In [3]:
from neo4j import GraphDatabase, basic_auth
import openai

driver = GraphDatabase.driver(
  "neo4j://18.212.176.170:7687",
  auth=basic_auth("neo4j", "menus-classifications-smell"))

```cypher
MATCH (m:Movie {title:$movie})<-[:RATED]-(u:User)-[:RATED]->(rec:Movie)
RETURN distinct rec.title AS recommendation LIMIT 20
```

- `(m:Movie {title:$movie})` : title > $movie
- `(rec:Movie)` : Variable > rec
- `RETURN distinct rec.title` : RETURN Title on variable rec



In [4]:
cypher_query = '''
MATCH (m:Movie {title:$movie})<-[:RATED]-(u:User)-[:RATED]->(rec:Movie)
RETURN distinct rec.title AS recommendation LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      movie="Crimson Tide").data())
  for record in results:
    print(record['recommendation'])

#driver.close()

/tmp/ipython-input-4-1491241831.py:7: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


Mr. Holland's Opus
Apollo 13
Dead Man Walking
Seven (a.k.a. Se7en)
Heat
Get Shorty
Fugitive, The
Dave
Addams Family Values
True Lies
Speed
Lion King, The
Four Weddings and a Funeral
Forrest Gump
Star Trek: Generations
Shawshank Redemption, The
Stargate
Pulp Fiction
Outbreak
Miracle on 34th Street


## 2. Plot Embedding

- Retriever goal : Search by plot to find similar movies

How? Vector retriever

### 2-1. Text to Embedding vector (OpenAI Embedding model)

In [5]:
def generate_embedding(text):
    embedding = openai.embeddings.create(input = [text], model='text-embedding-ada-002').data[0].embedding
    return embedding

In [7]:
def add_embedding_to_movie(tx):
    """Embed the plot of all Movie nodes and add the value to the embedding property."""
    result = tx.run("MATCH (m:Movie) WHERE m.plot IS NOT NULL RETURN m.title AS title, m.plot AS plot, ID(m) AS id LIMIT 100")
    cnt = 0
    for record in result:
        cnt += 1
        print(cnt)
        title = record["title"]
        plot = record["plot"]
        node_id = record["id"]
        print(plot)
        print("=================")
        embedding = generate_embedding(plot)
        # Save embedding vectors to Neo4j
        tx.run("MATCH (m:Movie) WHERE ID(m) = $id SET m.plotEmbedding = $embedding", id=node_id, embedding=embedding)
        print(f"Updated movie '{title}' with embedding.")

with driver.session() as session:
    session.write_transaction(add_embedding_to_movie)

/tmp/ipython-input-7-1372539816.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_embedding_to_movie)


1
A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.


Updated movie 'Toy Story' with embedding.
2
When two kids find and play a magical board game, they release a man trapped for decades in it and a host of dangers that can only be stopped by finishing the game.


Updated movie 'Jumanji' with embedding.
3
John and Max resolve to save their beloved bait shop from turning into an Italian restaurant, just as its new female owner catches Max's attention.


Updated movie 'Grumpier Old Men' with embedding.
4
Based on Terry McMillan's novel, this film follows four very different African-American women and their relationships with the male gender.


...

Updated movie 'City Hall' with embedding.
94
Focusing on a trio of friends and their elaborate plan to pull off a simple robbery and go on the run.
Updated movie 'Bottle Rocket' with embedding.
95
A single and lonely woman finds the seemingly perfect man to date, but soon regrets it when his deranged and possessive other personality emerges and worst still, she cannot convince anyone else of his Jekyll/Hyde true nature.


Updated movie 'Mr. Wrong' with embedding.
96
Unsuccessfully framed for his wife's murder, Dr. David Krane attempts to find the real culprit by utilizing a new drug that allows him to experience the memories of other people first-hand....
Updated movie 'Unforgettable' with embedding.
97
A rejected hockey player puts his skills to the golf course to save his grandmother's house.


Updated movie 'Happy Gilmore' with embedding.
98
Photographer Robert Kincaid wanders into the life of housewife Francesca Johnson, for four days in the 1960s.
Updated movie 'Bridges of Madison County, The' with embedding.
99
The Muppets' twist on the classic tale.


Updated movie 'Muppet Treasure Island' with embedding.
100
A camera follows model Christy Turlington through the spring fashion shows in Milan, Paris, and New York one year in the early 1990s, probably 1992. She and others dash from one designer's ...


Updated movie 'Catwalk' with embedding.


## 3. Generating a Retriever with Vector INDEX

Embedding model : https://platform.openai.com/docs/models/embeddings

VECTOR INDEX : https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/vector-indexes/

Genereate VECTOR INDEX
```cypher
CREATE VECTOR INDEX moviePlotsEmbedding FOR (n:Movie) ON (n.plotEmbedding) OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}
```

In [9]:
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")
retriever = VectorRetriever(
    driver,
    index_name="moviePlotsEmbedding",
    embedder=embedder,
    return_properties=["title", "plot"],
)

In [10]:
query_text = "A cowboy doll is jealous when a new spaceman figure becomes the top toy." # TOy Story Plot
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

items=[RetrieverResultItem(content='{\'title\': \'Toy Story\', \'plot\': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy\'s room."}', metadata={'score': 0.98028564453125, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:0'}), RetrieverResultItem(content="{'title': 'Indian in the Cupboard, The', 'plot': 'On his ninth birthday a boy receives many presents. Two of them first seem to be less important: an old cupboard from his brother and a little Indian figure made of plastic from his best ...'}", metadata={'score': 0.904388427734375, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:73'}), RetrieverResultItem(content="{'title': 'Powder', 'plot': 'A young bald albino boy with unique powers shakes up the rural community he lives in.'}", metadata={'score': 0.899505615234375, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:35'})] metadata={'query_vector': [-0.035

In [11]:
retriever_result

RetrieverResult(items=[RetrieverResultItem(content='{\'title\': \'Toy Story\', \'plot\': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy\'s room."}', metadata={'score': 0.98028564453125, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:0'}), RetrieverResultItem(content="{'title': 'Indian in the Cupboard, The', 'plot': 'On his ninth birthday a boy receives many presents. Two of them first seem to be less important: an old cupboard from his brother and a little Indian figure made of plastic from his best ...'}", metadata={'score': 0.904388427734375, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:73'}), RetrieverResultItem(content="{'title': 'Powder', 'plot': 'A young bald albino boy with unique powers shakes up the rural community he lives in.'}", metadata={'score': 0.899505615234375, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:35'})], metadata={'query

In [12]:
query_text = "A movie about a shooting incident."
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

items=[RetrieverResultItem(content="{'title': 'City Hall', 'plot': 'The accidental shooting of a boy in New York leads to an investigation by the Deputy Mayor, and unexpectedly far-reaching consequences.'}", metadata={'score': 0.93072509765625, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:109'}), RetrieverResultItem(content="{'title': 'Usual Suspects, The', 'plot': 'A sole survivor tells of the twisty events leading up to a horrific gun battle on a boat, which begin when five criminals meet at a seemingly random police lineup.'}", metadata={'score': 0.930206298828125, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:65'}), RetrieverResultItem(content="{'title': 'Nick of Time', 'plot': 'An unimpressive, every-day man is forced into a situation where he is told to kill a politician to save his kidnapped daughter.'}", metadata={'score': 0.9132537841796875, 'nodeLabels': ['Movie'], 'id': '4:69ab0fa5-25bf-4962-9594-4278a50ea22f:100'})] metadat

In [14]:
for k, item in enumerate(retriever_result.items):
    print(f"TOP {k+1}")
    print(item.content)
    print("score : ", item.metadata["score"])

TOP 1
{'title': 'City Hall', 'plot': 'The accidental shooting of a boy in New York leads to an investigation by the Deputy Mayor, and unexpectedly far-reaching consequences.'}
score :  0.93072509765625
TOP 2
{'title': 'Usual Suspects, The', 'plot': 'A sole survivor tells of the twisty events leading up to a horrific gun battle on a boat, which begin when five criminals meet at a seemingly random police lineup.'}
score :  0.930206298828125
TOP 3
{'title': 'Nick of Time', 'plot': 'An unimpressive, every-day man is forced into a situation where he is told to kill a politician to save his kidnapped daughter.'}
score :  0.9132537841796875
